In [7]:
from difflib import SequenceMatcher
from asrtoolkit import cer, wer
import cv2
import pytesseract
import numpy as np

In [ ]:
def ratio(a, b):
    # read more: https://docs.python.org/3/library/difflib.html#difflib.SequenceMatcher.ratio

    ratio1 = SequenceMatcher(None, a, b).ratio()
    ratio2 = SequenceMatcher(None, b, a).ratio()

    return max(ratio1, ratio2) * 100

In [10]:
fileName = '011'

img = cv2.imread('test_images/captured/grayscale/' + fileName + '_CC_GRAY.jpg', cv2.IMREAD_GRAYSCALE)
binary = cv2.imread('test_images/captured/binarized/' + fileName + '_SCANNED_BIN_GT.png', cv2.IMREAD_GRAYSCALE)

In [14]:
# img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
gray = cv2.medianBlur(img, 5)
edged = cv2.Canny(gray, 75, 200)


cv2.imwrite('origin.jpg', gray)

True

In [ ]:
textProcess = pytesseract.image_to_string(img, config = r'--oem 3 --psm 6')
textBinary = pytesseract.image_to_string(binary)

with open('test_images/truth/' + fileName + '_OCR_ASCII_TEXT_GT.txt', 'r') as f:
    textTruth = f.read()

with open('log/' + fileName + 'PCS.txt', 'w') as f:
    f.writelines(textProcess)
with open('log/' + fileName + 'BIN.txt', 'w') as f:
    f.writelines(textBinary)

In [ ]:
print(cer(textTruth, textProcess), cer(textTruth, textBinary))
print(wer(textTruth, textProcess), wer(textTruth, textBinary))

In [6]:


import math
from typing import Tuple, Union

import cv2
import numpy as np

from deskew import determine_skew


def rotate(
        image: np.ndarray, angle: float, background: Union[int, Tuple[int, int, int]]
) -> np.ndarray:
    old_width, old_height = image.shape[:2]
    angle_radian = math.radians(angle)
    width = abs(np.sin(angle_radian) * old_height) + abs(np.cos(angle_radian) * old_width)
    height = abs(np.sin(angle_radian) * old_width) + abs(np.cos(angle_radian) * old_height)

    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    rot_mat[1, 2] += (width - old_width) / 2
    rot_mat[0, 2] += (height - old_height) / 2
    return cv2.warpAffine(image, rot_mat, (int(round(height)), int(round(width))), borderValue=background)

image = cv2.imread('test_images/captured/grayscale/011_CC_GRAY.jpg')
grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
angle = determine_skew(grayscale)
rotated = rotate(image, angle, (0, 0, 0))
cv2.imwrite('output.png', rotated)

True